In [2]:
from vision.tracker import Tracker, get_marker_object, get_markers

In [4]:
vision_thread = Tracker(mid=5,
                        transform=None,
                        mid_aux=0,
                        video_source=-1,
                        capture=False,
                        show=True,
                        debug=False,
                       )

starting Tracker, video source:  -1


In [5]:
vision_thread.start()

Last markers len: 8
6
Restore Calibration


In [18]:
robot = get_marker_object(8)
dummy = get_marker_object(7)

In [19]:
robot.realxy()


array([0.5938483 , 0.47939653, 0.        ])

In [20]:
dummy.realxy()

array([0.59646187, 0.38297419, 0.        ])

In [21]:
from vision.tracker import euclidian_distance

In [22]:
dist = euclidian_distance(robot.realxy()[:2], dummy.realxy()[:2])

In [25]:
dummy.orientation()

1.7359450042095235